# Kapitola 28: Uvod do neuronovych siti - Postavte si svuj prvni umely neuron

---

## Co se naucite

V teto kapitole se naucite:
- Pochopit **biologickou inspiraci** neuronovych siti
- Jak funguje **Perceptron** - nejjednodussi umely neuron
- Co jsou **vahy, bias a aktivacni funkce**
- Naprogramovat Perceptron **od nuly** v Pythonu
- Naucit neuron resit **logicke brany AND a OR**
- Vizualizovat **rozhodovaci hranici**

## Proc neuronove site?

Neuronove site jsou zakladem moderni umele inteligence. Vsechny velke modely jako ChatGPT,
rozpoznavani obrazu ci autonomni auta jsou postaveny na neuronovych sitich.

Dnes si postavime ten nejzakladnejsi stavebni kamen - **Perceptron**.

## 1. Instalace a import knihoven

In [ ]:
# Instalace potrebnych knihoven
!pip install numpy matplotlib -q

print("Knihovny nainstalovany!")

In [ ]:
# Import knihoven
import numpy as np
import matplotlib.pyplot as plt

# Nastaveni grafu
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = [10, 6]

# Pro reprodukovatelnost
np.random.seed(42)

print("NumPy verze:", np.__version__)
print("Prostredi pripraveno!")

## 2. Biologicka inspirace - Jak funguje mozek?

Lidsky mozek obsahuje asi **86 miliard neuronu**. Kazdy neuron:

1. **Prijima signaly** od jinych neuronu pres dendrity
2. **Zpracovava signaly** v tele bunky
3. **Vysilat signal** dal pres axon, pokud je podnět dostatečně silný

Umely neuron (Perceptron) tento proces zjednodusuje:

```
Vstupy (x1, x2, ...) --> [Vazeny soucet] --> [Aktivacni funkce] --> Vystup (0 nebo 1)
```

In [ ]:
# Vizualizace umeleho neuronu
fig, ax = plt.subplots(figsize=(12, 6))

# Skryt osy
ax.axis('off')

# Vstupy
ax.annotate('x1', xy=(0.1, 0.7), fontsize=16, ha='center', fontweight='bold')
ax.annotate('x2', xy=(0.1, 0.5), fontsize=16, ha='center', fontweight='bold')
ax.annotate('x3', xy=(0.1, 0.3), fontsize=16, ha='center', fontweight='bold')

# Vahy
ax.annotate('w1', xy=(0.25, 0.65), fontsize=12, color='blue')
ax.annotate('w2', xy=(0.25, 0.5), fontsize=12, color='blue')
ax.annotate('w3', xy=(0.25, 0.35), fontsize=12, color='blue')

# Sipky od vstupu k neuronu
for y in [0.7, 0.5, 0.3]:
    ax.annotate('', xy=(0.4, 0.5), xytext=(0.15, y),
               arrowprops=dict(arrowstyle='->', color='gray', lw=2))

# Neuron (kruh)
circle = plt.Circle((0.5, 0.5), 0.1, color='lightblue', ec='darkblue', lw=3)
ax.add_patch(circle)
ax.annotate('Σ + b', xy=(0.5, 0.5), fontsize=14, ha='center', va='center', fontweight='bold')

# Sipka k aktivacni funkci
ax.annotate('', xy=(0.7, 0.5), xytext=(0.6, 0.5),
           arrowprops=dict(arrowstyle='->', color='gray', lw=2))

# Aktivacni funkce
rect = plt.Rectangle((0.7, 0.4), 0.1, 0.2, color='lightyellow', ec='orange', lw=2)
ax.add_patch(rect)
ax.annotate('f(x)', xy=(0.75, 0.5), fontsize=12, ha='center', va='center')

# Vystup
ax.annotate('', xy=(0.92, 0.5), xytext=(0.8, 0.5),
           arrowprops=dict(arrowstyle='->', color='gray', lw=2))
ax.annotate('Vystup\n(0 nebo 1)', xy=(0.95, 0.5), fontsize=14, ha='left', va='center', fontweight='bold')

# Popisky
ax.annotate('VSTUPY', xy=(0.1, 0.85), fontsize=12, ha='center', style='italic')
ax.annotate('VAHY', xy=(0.25, 0.85), fontsize=12, ha='center', style='italic', color='blue')
ax.annotate('VAZENY\nSOUCET', xy=(0.5, 0.25), fontsize=10, ha='center', style='italic')
ax.annotate('AKTIVACNI\nFUNKCE', xy=(0.75, 0.25), fontsize=10, ha='center', style='italic', color='orange')

plt.title('Struktura umeleho neuronu (Perceptronu)', fontsize=16, fontweight='bold', pad=20)
plt.xlim(0, 1.1)
plt.ylim(0, 1)
plt.tight_layout()
plt.show()

print("\nVzorec Perceptronu:")
print("vystup = f(w1*x1 + w2*x2 + w3*x3 + bias)")
print("\nkde f(x) je aktivacni funkce (napr. skokova: 1 pokud x >= 0, jinak 0)")

## 3. Matematika Perceptronu

Perceptron pracuje v techto krocich:

### Krok 1: Vazeny soucet
$$z = \sum_{i=1}^{n} w_i \cdot x_i + b = w_1 x_1 + w_2 x_2 + ... + w_n x_n + b$$

### Krok 2: Aktivacni funkce (skokova)
$$y = f(z) = \begin{cases} 1 & \text{pokud } z \geq 0 \\ 0 & \text{pokud } z < 0 \end{cases}$$

### Krok 3: Uceni (aktualizace vah)
$$w_i = w_i + \eta \cdot (y_{skutecne} - y_{predikovane}) \cdot x_i$$

kde $\eta$ je **rychlost uceni (learning rate)**

In [ ]:
# Demonstrace skokove aktivacni funkce
def step_function(x):
    """Skokova aktivacni funkce"""
    return np.where(x >= 0, 1, 0)

# Vytvoreni dat pro vizualizaci
x = np.linspace(-5, 5, 100)
y = step_function(x)

# Vizualizace
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.plot(x, y, 'b-', linewidth=2)
plt.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
plt.axvline(x=0, color='gray', linestyle='--', alpha=0.5)
plt.scatter([0], [1], color='blue', s=100, zorder=5)  # Bod v 0 je 1
plt.title('Skokova aktivacni funkce (Step)', fontweight='bold')
plt.xlabel('Vazeny soucet (z)')
plt.ylabel('Vystup')
plt.ylim(-0.2, 1.2)
plt.grid(True, alpha=0.3)

# Pro porovnani - sigmoid (pouziva se v modernich sitich)
plt.subplot(1, 2, 2)
sigmoid = 1 / (1 + np.exp(-x))
plt.plot(x, sigmoid, 'r-', linewidth=2)
plt.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5)
plt.axvline(x=0, color='gray', linestyle='--', alpha=0.5)
plt.title('Sigmoid aktivacni funkce (moderni site)', fontweight='bold')
plt.xlabel('Vazeny soucet (z)')
plt.ylabel('Vystup')
plt.ylim(-0.2, 1.2)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Skokova funkce: Vraci 0 nebo 1 (ostre rozhodnuti)")
print("Sigmoid: Vraci hodnotu mezi 0 a 1 (plynule rozhodnuti)")

## 4. Implementace Perceptronu od nuly

Nyni naprogramujeme kompletni Perceptron v Pythonu.

In [ ]:
class Perceptron:
    """
    Perceptron - nejjednodussi umely neuron
    
    Parametry:
    - learning_rate: Rychlost uceni (jak moc menime vahy pri kazde chybe)
    - n_iterations: Pocet pruchodu cely dataset
    """
    
    def __init__(self, learning_rate=0.1, n_iterations=100):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.weights = None
        self.bias = None
        self.errors_history = []  # Pro sledovani uceni
    
    def _step_function(self, x):
        """Skokova aktivacni funkce"""
        return np.where(x >= 0, 1, 0)
    
    def fit(self, X, y):
        """
        Natrenuje Perceptron na datech
        
        X: Vstupni data (matice n_vzorku x n_features)
        y: Cilove hodnoty (vektor 0 a 1)
        """
        n_samples, n_features = X.shape
        
        # Inicializace vah na nuly
        self.weights = np.zeros(n_features)
        self.bias = 0
        
        # Ucici cyklus
        for iteration in range(self.n_iterations):
            errors = 0
            
            for idx, x_i in enumerate(X):
                # 1. Vypocet vazeneho souctu
                linear_output = np.dot(x_i, self.weights) + self.bias
                
                # 2. Aplikace aktivacni funkce
                y_predicted = self._step_function(linear_output)
                
                # 3. Vypocet chyby
                error = y[idx] - y_predicted
                
                # 4. Aktualizace vah pokud je chyba
                if error != 0:
                    update = self.learning_rate * error
                    self.weights += update * x_i
                    self.bias += update
                    errors += 1
            
            self.errors_history.append(errors)
            
            # Pokud neni zadna chyba, muzeme skoncit drive
            if errors == 0:
                print(f"Perceptron se naucil po {iteration + 1} iteracich!")
                break
        
        return self
    
    def predict(self, X):
        """Predikce pro nova data"""
        linear_output = np.dot(X, self.weights) + self.bias
        return self._step_function(linear_output)
    
    def score(self, X, y):
        """Presnost modelu"""
        predictions = self.predict(X)
        return np.mean(predictions == y)

print("Trida Perceptron definovana!")
print("\nKlicove metody:")
print("- fit(X, y): Natrenuje model na datech")
print("- predict(X): Vraci predikce")
print("- score(X, y): Vraci presnost")

## 5. Logicka brana AND - Prvni uloha pro Perceptron

Logicka brana AND vraci 1 pouze pokud OBE vstupy jsou 1:

| Vstup 1 | Vstup 2 | AND |
|---------|---------|-----|
| 0 | 0 | 0 |
| 0 | 1 | 0 |
| 1 | 0 | 0 |
| 1 | 1 | 1 |

In [ ]:
# Data pro logickou branu AND
X_and = np.array([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1]
])

y_and = np.array([0, 0, 0, 1])  # AND: 1 pouze kdyz oba vstupy jsou 1

print("Trenovaci data pro AND:")
print("Vstup 1 | Vstup 2 | Vystup (AND)")
print("-" * 35)
for i in range(len(X_and)):
    print(f"   {X_and[i, 0]}    |    {X_and[i, 1]}    |      {y_and[i]}")

In [ ]:
# Vytvoreni a trenovani Perceptronu pro AND
perceptron_and = Perceptron(learning_rate=0.1, n_iterations=20)
perceptron_and.fit(X_and, y_and)

print("\n" + "=" * 50)
print("NAUCENE PARAMETRY:")
print("=" * 50)
print(f"Vahy: w1 = {perceptron_and.weights[0]:.2f}, w2 = {perceptron_and.weights[1]:.2f}")
print(f"Bias: b = {perceptron_and.bias:.2f}")
print(f"\nRovnice rozhodovaci hranice:")
print(f"{perceptron_and.weights[0]:.2f}*x1 + {perceptron_and.weights[1]:.2f}*x2 + {perceptron_and.bias:.2f} = 0")

In [ ]:
# Testovani Perceptronu
print("TESTOVANI PERCEPTRONU PRO AND:")
print("=" * 40)
print("Vstup    | Ocekavano | Predikovano | Spravne?")
print("-" * 40)

predictions = perceptron_and.predict(X_and)
for i in range(len(X_and)):
    correct = "ANO" if predictions[i] == y_and[i] else "NE"
    print(f"[{X_and[i, 0]}, {X_and[i, 1]}]   |     {y_and[i]}     |      {predictions[i]}      |   {correct}")

print("-" * 40)
print(f"Presnost: {perceptron_and.score(X_and, y_and) * 100:.0f}%")

In [ ]:
# Vizualizace rozhodovaci hranice
def plot_decision_boundary(perceptron, X, y, title):
    """Vizualizuje rozhodovaci hranici Perceptronu"""
    plt.figure(figsize=(10, 8))
    
    # Barvy pro jednotlive tridy
    colors = ['red' if label == 0 else 'blue' for label in y]
    
    # Body
    plt.scatter(X[:, 0], X[:, 1], c=colors, s=200, edgecolors='black', linewidths=2, zorder=5)
    
    # Popisky bodu
    for i, (x1, x2) in enumerate(X):
        plt.annotate(f'({x1},{x2})\ny={y[i]}', 
                    xy=(x1, x2), 
                    xytext=(x1+0.1, x2+0.15),
                    fontsize=10)
    
    # Rozhodovaci hranice (pokud existuji nenulove vahy)
    if perceptron.weights[1] != 0:
        x1_line = np.linspace(-0.5, 1.5, 100)
        # z rovnice: w1*x1 + w2*x2 + b = 0
        # x2 = (-w1*x1 - b) / w2
        x2_line = (-perceptron.weights[0] * x1_line - perceptron.bias) / perceptron.weights[1]
        plt.plot(x1_line, x2_line, 'g-', linewidth=3, label='Rozhodovaci hranice')
        
        # Vyplneni oblasti
        plt.fill_between(x1_line, x2_line, 2, alpha=0.1, color='blue', label='Oblast 1')
        plt.fill_between(x1_line, x2_line, -1, alpha=0.1, color='red', label='Oblast 0')
    
    plt.xlim(-0.5, 1.5)
    plt.ylim(-0.5, 1.5)
    plt.xlabel('Vstup 1 (x1)', fontsize=12)
    plt.ylabel('Vstup 2 (x2)', fontsize=12)
    plt.title(title, fontsize=14, fontweight='bold')
    plt.legend(loc='upper left')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# Vizualizace AND
plot_decision_boundary(perceptron_and, X_and, y_and, 'Perceptron - Logicka brana AND')

## 6. Logicka brana OR

OR vraci 1 pokud ALESPON JEDEN vstup je 1:

| Vstup 1 | Vstup 2 | OR |
|---------|---------|----|
| 0 | 0 | 0 |
| 0 | 1 | 1 |
| 1 | 0 | 1 |
| 1 | 1 | 1 |

In [ ]:
# Data pro logickou branu OR
X_or = np.array([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1]
])

y_or = np.array([0, 1, 1, 1])  # OR: 1 kdyz alespon jeden vstup je 1

# Trenovani
perceptron_or = Perceptron(learning_rate=0.1, n_iterations=20)
perceptron_or.fit(X_or, y_or)

print(f"\nNaucene vahy: w1 = {perceptron_or.weights[0]:.2f}, w2 = {perceptron_or.weights[1]:.2f}")
print(f"Bias: b = {perceptron_or.bias:.2f}")
print(f"Presnost: {perceptron_or.score(X_or, y_or) * 100:.0f}%")

In [ ]:
# Vizualizace OR
plot_decision_boundary(perceptron_or, X_or, y_or, 'Perceptron - Logicka brana OR')

## 7. Problem XOR - Limitace Perceptronu

**XOR** (exclusive or) vraci 1 pouze kdyz vstupy jsou **ruzne**:

| Vstup 1 | Vstup 2 | XOR |
|---------|---------|-----|
| 0 | 0 | 0 |
| 0 | 1 | 1 |
| 1 | 0 | 1 |
| 1 | 1 | 0 |

**Perceptron tuto ulohu NEZVLADNE!** Proc?

In [ ]:
# Data pro XOR
X_xor = np.array([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1]
])

y_xor = np.array([0, 1, 1, 0])  # XOR: 1 pouze kdyz vstupy jsou ruzne

# Pokus o trenovani
perceptron_xor = Perceptron(learning_rate=0.1, n_iterations=100)
perceptron_xor.fit(X_xor, y_xor)

predictions = perceptron_xor.predict(X_xor)
print(f"\nPresnost na XOR: {perceptron_xor.score(X_xor, y_xor) * 100:.0f}%")
print(f"Predikce: {predictions}")
print(f"Ocekavano: {y_xor}")

In [ ]:
# Vizualizace problemu XOR
plt.figure(figsize=(10, 8))

# Body
colors = ['red' if label == 0 else 'blue' for label in y_xor]
plt.scatter(X_xor[:, 0], X_xor[:, 1], c=colors, s=300, edgecolors='black', linewidths=2, zorder=5)

# Popisky
labels = ['(0,0)\nXOR=0', '(0,1)\nXOR=1', '(1,0)\nXOR=1', '(1,1)\nXOR=0']
positions = [(0.15, 0.1), (-0.15, 1.1), (1.15, 0.1), (1.15, 1.1)]
for i, (label, pos) in enumerate(zip(labels, positions)):
    plt.annotate(label, xy=(X_xor[i, 0], X_xor[i, 1]), xytext=pos, fontsize=11)

# Priklad ruznych primek - zadna nefunguje
x_line = np.linspace(-0.5, 1.5, 100)
plt.plot(x_line, 0.5 - x_line, 'g--', alpha=0.5, label='Pokus 1: nefunguje')
plt.plot(x_line, x_line, 'orange', linestyle='--', alpha=0.5, label='Pokus 2: nefunguje')
plt.axhline(y=0.5, color='purple', linestyle='--', alpha=0.5, label='Pokus 3: nefunguje')

plt.xlim(-0.5, 1.5)
plt.ylim(-0.5, 1.5)
plt.xlabel('Vstup 1', fontsize=12)
plt.ylabel('Vstup 2', fontsize=12)
plt.title('Problem XOR - nelze oddelit jednou primkou!', fontsize=14, fontweight='bold')
plt.legend(loc='upper right')
plt.grid(True, alpha=0.3)

# Textovy popis
plt.text(0.5, -0.35, 'XOR problem: Cervene (0) a modre (1) body nelze oddelit\njedinou primkou = problem neni LINEARNI SEPAROVATELNY', 
        ha='center', fontsize=11, style='italic', color='darkred')

plt.tight_layout()
plt.show()

print("\nPROBLEM XOR:")
print("Perceptron dokaze resit pouze LINEARNI SEPAROVATELNE problemy.")
print("XOR vyzaduje VICEVRSTVE neuronove site (MLP - Multi-Layer Perceptron).")

## 8. Historie uceni - Jak Perceptron konverguje

In [ ]:
# Vizualizace procesu uceni
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(perceptron_and.errors_history, 'b-o', linewidth=2, markersize=8)
plt.title('Uceni Perceptronu - AND', fontweight='bold')
plt.xlabel('Iterace')
plt.ylabel('Pocet chyb')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(perceptron_or.errors_history, 'r-o', linewidth=2, markersize=8)
plt.title('Uceni Perceptronu - OR', fontweight='bold')
plt.xlabel('Iterace')
plt.ylabel('Pocet chyb')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Graf ukazuje, jak rychle se Perceptron nauci (chyby klesaji k 0).")
print(f"AND: Konvergoval po {len(perceptron_and.errors_history)} iteracich")
print(f"OR: Konvergoval po {len(perceptron_or.errors_history)} iteracich")

## 9. Mini-projekt: Klasifikace bodu v rovine

Vytvorime vetsi dataset a natrenovame Perceptron na realnejsich datech.

In [ ]:
# Generovani syntetickych dat
np.random.seed(42)

# Trida 0: Body v levem dolnim rohu
X_class0 = np.random.randn(50, 2) * 0.5 + np.array([1, 1])
y_class0 = np.zeros(50)

# Trida 1: Body v pravem hornim rohu
X_class1 = np.random.randn(50, 2) * 0.5 + np.array([3, 3])
y_class1 = np.ones(50)

# Spojeni
X = np.vstack([X_class0, X_class1])
y = np.hstack([y_class0, y_class1]).astype(int)

# Promichani
shuffle_idx = np.random.permutation(len(X))
X = X[shuffle_idx]
y = y[shuffle_idx]

print(f"Dataset: {len(X)} vzorku")
print(f"Trida 0: {sum(y == 0)} vzorku")
print(f"Trida 1: {sum(y == 1)} vzorku")

In [ ]:
# Rozdeleni na trenovaci a testovaci data (rucne, bez sklearn)
split_idx = int(0.8 * len(X))
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

print(f"Trenovaci data: {len(X_train)} vzorku")
print(f"Testovaci data: {len(X_test)} vzorku")

In [ ]:
# Trenovani Perceptronu
perceptron = Perceptron(learning_rate=0.01, n_iterations=100)
perceptron.fit(X_train, y_train)

# Vyhodnoceni
train_accuracy = perceptron.score(X_train, y_train)
test_accuracy = perceptron.score(X_test, y_test)

print(f"\nVYSLEDKY:")
print(f"Trenovaci presnost: {train_accuracy * 100:.1f}%")
print(f"Testovaci presnost: {test_accuracy * 100:.1f}%")
print(f"\nNaucene vahy: {perceptron.weights}")
print(f"Bias: {perceptron.bias}")

In [ ]:
# Vizualizace vysledku
plt.figure(figsize=(12, 5))

# Data a rozhodovaci hranice
plt.subplot(1, 2, 1)
colors = ['red' if label == 0 else 'blue' for label in y]
plt.scatter(X[:, 0], X[:, 1], c=colors, alpha=0.6, edgecolors='black', s=50)

# Rozhodovaci hranice
if perceptron.weights[1] != 0:
    x_line = np.linspace(X[:, 0].min() - 1, X[:, 0].max() + 1, 100)
    y_line = (-perceptron.weights[0] * x_line - perceptron.bias) / perceptron.weights[1]
    plt.plot(x_line, y_line, 'g-', linewidth=3, label='Rozhodovaci hranice')

plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Klasifikace bodu v rovine', fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

# Historie uceni
plt.subplot(1, 2, 2)
plt.plot(perceptron.errors_history, 'b-', linewidth=2)
plt.xlabel('Iterace')
plt.ylabel('Pocet chyb')
plt.title('Konvergence uceni', fontweight='bold')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 10. Shrnuti a klicove poznatky

### Co jsme se naucili:

| Koncept | Popis |
|---------|-------|
| **Perceptron** | Nejjednodussi umely neuron |
| **Vahy (weights)** | Dulezitost jednotlivych vstupu |
| **Bias** | Posunuti rozhodovaci hranice |
| **Aktivacni funkce** | Rozhoduje o vystupu (0 nebo 1) |
| **Uceni** | Uprava vah podle chyb |
| **Linearni separabilita** | Perceptron dokaze oddelit pouze linearni problemy |

### Klicove poznatky:

1. **Perceptron je zakladem** vsech neuronovych siti
2. **Umi resit** linearni separovatelne problemy (AND, OR)
3. **Neumi resit** nelinearni problemy (XOR)
4. **Pro slozitejsi ulohy** potrebujeme vicevrstve site
5. **Moderni site** pouzivaji plynule aktivacni funkce (sigmoid, ReLU)

## 11. Kviz: Otestujte sve znalosti

In [ ]:
def kviz_perceptron():
    print("=" * 60)
    print("KVIZ: Perceptron a neuronove site")
    print("=" * 60)
    
    otazky = [
        {
            "otazka": "Co je hlavni funkci vah (weights) v Perceptronu?",
            "moznosti": ["a) Urci barvu vystupu", "b) Urci dulezitost vstupu", "c) Urci rychlost uceni", "d) Urci pocet iteraci"],
            "spravne": "b",
            "vysvetleni": "Vahy urcuji, jak dulezity je kazdy vstup pro konecne rozhodnuti."
        },
        {
            "otazka": "Co dela skokova aktivacni funkce?",
            "moznosti": ["a) Vraci vzdy 0.5", "b) Vraci 0 nebo 1 podle prahu", "c) Nasobi vstup vahou", "d) Pocita prumer"],
            "spravne": "b",
            "vysvetleni": "Skokova funkce vraci 1 pokud vstup >= 0, jinak vraci 0."
        },
        {
            "otazka": "Kterou logickou branu Perceptron NEMUZE vyresit?",
            "moznosti": ["a) AND", "b) OR", "c) XOR", "d) NOT"],
            "spravne": "c",
            "vysvetleni": "XOR neni linearni separovatelny problem - nelze ho oddelit jednou primkou."
        },
        {
            "otazka": "Co je bias v Perceptronu?",
            "moznosti": ["a) Typ aktivacni funkce", "b) Posunuti rozhodovaci hranice", "c) Chyba modelu", "d) Pocet vstupu"],
            "spravne": "b",
            "vysvetleni": "Bias posouvá rozhodovaci hranici, umoznuje lepsi fitovani dat."
        },
        {
            "otazka": "Jak se Perceptron uci?",
            "moznosti": ["a) Nahodnym generovanim vah", "b) Upravou vah podle chyb", "c) Kopirovanim dat", "d) Neuci se"],
            "spravne": "b",
            "vysvetleni": "Perceptron upravuje vahy smerem, ktery snizuje chybu predikce."
        }
    ]
    
    skore = 0
    
    for i, q in enumerate(otazky, 1):
        print(f"\n{i}. {q['otazka']}")
        for m in q['moznosti']:
            print(f"   {m}")
        
        odpoved = input("   Vase odpoved (a/b/c/d): ").strip().lower()
        
        if odpoved == q['spravne']:
            print("   Spravne!")
            skore += 1
        else:
            print(f"   Spatne. Spravna odpoved je {q['spravne']})")
        print(f"   Vysvetleni: {q['vysvetleni']}")
    
    print("\n" + "=" * 60)
    print(f"VYSLEDEK: {skore}/{len(otazky)} spravnych odpovedi")
    
    if skore == len(otazky):
        print("Vyborne! Rozumite zakladum neuronovych siti!")
    elif skore >= len(otazky) * 0.6:
        print("Dobre! Mate solidni zaklady.")
    else:
        print("Zkuste si kapitolu projit znovu.")

# Spustit kviz
kviz_perceptron()

## 12. Vyzva pro vas

Zkuste nasledujici experimenty:

1. **Zmena learning rate**: Co se stane kdyz pouzijete `learning_rate=0.5` nebo `learning_rate=0.001`?
2. **Brana NAND**: Vytvorte data pro NAND branu (opak AND) a natrenovejte Perceptron
3. **Vlastni data**: Vygenerujte vlastni 2D data a zkuste je klasifikovat

In [ ]:
# Prostor pro vase experimenty
# Zkuste napriklad:
# y_nand = np.array([1, 1, 1, 0])  # NAND brana



---

## Zaver

V teto kapitole jste:

- Pochopili biologickou inspiraci neuronovych siti
- Naprogramovali Perceptron od nuly
- Naucili neuron resit logicke brany AND a OR
- Pochopili limitace jednovrstvych siti (XOR problem)
- Vizualizovali rozhodovaci hranice

**V dalsi kapitole** se podivame na rozhodovaci stromy - dalsi duleziity algoritmus strojoveho uceni!

---
*Kapitola 28 | Uvod do neuronovych siti | AI Akademie*